In [2]:
%tensorflow_version 2.x

from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np


In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


In [6]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('lenght of test : {} characters'.format(len(text)))

lenght of test : 1115394 characters


In [7]:
## encoding the text, here we encode each character with an integer

vocab = sorted(set(text)) # sort all unique characters 

# create a mapping from unique characters to indices
char_to_index = {u:i for i,u in enumerate(vocab)} # assigns lets say 0 to to n number of unique words stored in the vocab
#reverse
index_to_char = np.array(vocab)

def text2int(text):
  return np.array([char_to_index[a] for a in text])

text_int = text2int(text)

In [14]:
print("text : ", text[:19])
text_int1= text2int(text[:19]) # example to test the conversion

print("Encoded text: ",text_int1)



text :  First Citizen:
Befo
Encoded text:  [18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53]


In [15]:
# if we were to go the other way around that is from int to text 

def int_to_text(integer):
  try:
    integer = integer.numpy()
  except:
    pass
  return "".join(index_to_char[integer])

print(int_to_text(text_int1)) # example to est the conversion 

First Citizen:
Befo


In [17]:
# create training examples 

sequence_length = 100 
ex_per_epoch =(len(text)//(sequence_length +1)) # we want the out put to contain the input word minus the first word and a new word at the end 

char_data = tf.data.Dataset.from_tensor_slices(text_int) # we want to slice the data int 101 words each.where 100 will be the training and 1 will be the precdiction 

seq = char_data.batch(sequence_length+1, drop_remainder=True)

In [20]:
# create the training example 

def split_target(bits):
  inputs = bits[:-1]
  labels = bits[1:]
  return inputs, labels 

datasets = seq.map(split_target)

In [ ]:
for a,b in datasets.take(2):
  print('\n\nExample\n')
  print("Input")
  print(int_to_text(a))
  print("\nOutput")
  print(int_to_text(b))

In [22]:
# make training batches 

batchSize =64
vocabSize = len(vocab)
embedding = 256

rnnUnit = 1024

##i just copied this from tim it makes sense but if you have a better way thats fine also share them 

bufferSize = 10000

data = datasets.shuffle(bufferSize).batch(batchSize, drop_remainder = True)


In [26]:
# building the model 
def model_build(vocabSize, embedding, rnnUnit,batchSize) :
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocabSize, embedding,batch_input_shape=[batchSize, None]),
                              tf.keras.layers.LSTM(rnnUnit, return_sequences= True,
                                                   stateful = True,
                                                   recurrent_initializer = 'glorot_uniform'),
                              tf.keras.layers.Dense(vocabSize
                                                    )
  ])
  return model




In [27]:
model = model_build(vocabSize, embedding, rnnUnit, batchSize)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_1 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________
